In [1]:
# Import libraries
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="school_analytics",
    user="ashleighboudier",
    password="YOUR_PASSWORD",  # <-- replace with your password
    host="localhost"
)

print("Connected to PostgreSQL!")


Matplotlib is building the font cache; this may take a moment.


Connected to PostgreSQL!


In [ ]:
# Query attendance data
attendance_query = """
SELECT 
    s.student_id,
    s.first_name || ' ' || s.last_name AS student_name,
    COUNT(*) FILTER (WHERE fa.attendance_status = 'Present')::decimal / COUNT(*) * 100 AS attendance_percentage
FROM fact_attendance fa
JOIN dim_student s ON fa.student_id = s.student_id
GROUP BY s.student_id, s.first_name, s.last_name
ORDER BY attendance_percentage DESC;
"""

attendance_df = pd.read_sql(attendance_query, conn)
attendance_df


In [ ]:
# Plot attendance %
plt.figure(figsize=(12,6))
plt.bar(attendance_df['student_name'], attendance_df['attendance_percentage'], color='skyblue')
plt.xticks(rotation=90)
plt.ylabel('Attendance %')
plt.title('Student Attendance Percentage')
plt.tight_layout()
plt.show()


In [ ]:
# Query grades
grades_query = """
SELECT 
    s.student_id,
    s.first_name || ' ' || s.last_name AS student_name,
    fg.term,
    AVG(fg.grade) AS avg_grade
FROM fact_grades fg
JOIN dim_student s ON fg.student_id = s.student_id
GROUP BY s.student_id, s.first_name, s.last_name, fg.term
ORDER BY fg.term, avg_grade DESC;
"""

grades_df = pd.read_sql(grades_query, conn)
grades_df


In [ ]:
# Plot grades per term
terms = grades_df['term'].unique()

for term in terms:
    term_df = grades_df[grades_df['term'] == term]
    plt.figure(figsize=(12,6))
    plt.bar(term_df['student_name'], term_df['avg_grade'], color='orange')
    plt.xticks(rotation=90)
    plt.ylabel('Average Grade')
    plt.title(f'Average Grades - Term {term}')
    plt.tight_layout()
    plt.show()


In [2]:
conn.close()
print("Database connection closed.")


Database connection closed.
